In [6]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
import re


In [79]:
df = pd.read_csv('non_fire.csv')
df.head(5)

,time,lat,lon,area,lst_day,lst_night,evi,class
0,2018-06-26,33.408854,-117.076050,1.373816e+07,307.487500,289.228750,0.216975,no_fire
1,2018-06-26,36.137575,-118.225236,9.530845e+07,306.894414,284.389730,0.234126,no_fire
2,2018-06-26,41.055741,-121.923551,5.452330e+08,300.717102,291.531937,0.398504,no_fire
3,2018-07-04,37.479687,-120.400798,4.121447e+07,322.606667,293.778333,0.166904,no_fire
4,2018-07-04,35.525000,-118.555102,5.151808e+06,307.930000,291.180000,0.362250,no_fire


In [101]:
def date_to_month(datetime):
    datetime = int(re.search("-\d\d",datetime).group(0))
    return datetime

In [105]:
df['month'] = abs(df.time.apply(date_to_month))

In [106]:
df.month.value_counts()

6     18
7     17
1     17
12    15
9     15
11    13
10    11
8     11
5     11
2     11
3     10
4      7
Name: month, dtype: int64

In [ ]:
y = df['class']
x = df.drop(['class','time'], axis=1)

In [122]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.25, random_state = 42)

In [128]:
#MLP Classifier Function
#solver: 'Adam' works better for bigger datasets
#regularization parameter: paper did between 0.8 - 4, and compared results
#hiddenlayers: 1 case did 2 hidden layers with 64 hidden units in each
#
clf = MLPClassifier(solver='adam', alpha=1, max_iter=1000,hidden_layer_sizes=(2,64), random_state=1)
clf.fit(train_x, train_y)

MLPClassifier(activation='relu', alpha=1, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(2, 64), learning_rate='constant',
              learning_rate_init=0.001, max_iter=1000, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [129]:
train_preds = clf.predict(train_x)
test_preds = clf.predict(test_x)